In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
#data = pd.read_excel('Data.xlsx')
mydata = pd.read_csv('/content/drive/My Drive/IIIT_A/DA/car.csv',delimiter=',')

# mydata =pd.read_csv('adult.csv', delimiter=',',header = None) 
mydata = pd.DataFrame(mydata)
# mydata = list(mydata)
mydata

,buying,maint,doors,persons,lug boot,safety,target
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [18]:
def entropy_profit(ref):
  unacc =0
  acc =0 
  good =0  
  vgood =0
  for i in ref:
    if(i == 'unacc' or i == 'acc'):
      acc = acc+1
    # elif (i == 'acc'):
    #  acc= acc+1
    # elif (i == 'good'):
    #  good= good+1
    else:
      vgood = vgood +1

  # prob_unacc = unacc/len(ref)
  prob_acc = acc/len(ref)
  # prob_vgood = vgood/len(ref)
  # prob_good = good/len(ref)
  
  
  return   prob_acc



def gain_ratio(a, length):
  return -((a/length)*(math.log2(a/length)))



def entropy(yes,no):
  if(no == 0):
      return -( (yes/(yes+no))*(math.log2(yes/(yes+no))))
  elif(yes == 0):
      return - (no/(yes+no))*(math.log2(no/(yes+no)))
  else:
      return -( (yes/(yes+no))*(math.log2(yes/(yes+no))) + (no/(yes+no))*(math.log2(no/(yes+no))) )




def Input_data(df1,move):
  
  df =  df1.drop('target',axis = 1)
  ref = df1['target']
  node_child = dict()
  row , col = df.shape
  length = len(list(df.columns))
  node =""
  val = -1.0
  for i in range(0,length):
    index =-1
    attri = list()
    count_attri =dict()
    for j in df.iloc[:,i]: 
      count_attri[j] = 0
    for j in df.iloc[:,i]:
      index = index + 1.0
      attri.append(j)

# prob_unacc , prob_acc, prob_vgood ,prob_good

      if(ref[index] == "unacc" or ref[index] == "acc"):
        if j in count_attri:
            count_attri[j] = count_attri[j] + 1.0

        else:
            count_attri[j] = 1.0          
  
    indi_entropy_element = dict()
    # print(attri)
    # print("\n")



    for ele in set(attri):
      if(type(ele) == float):
        no = df.iloc[:,i].value_counts()[int(ele)] - count_attri[int(ele)]
      else:
        no = df.iloc[:,i].value_counts()[ele] - count_attri[ele]   
        

      indi_entropy_element[ele] = abs(entropy(int(count_attri[ele]) , int(no)))

    
    S = entropy_profit(ref)


    for ele in set(attri):
      if(type(ele) == float):
        #print(input.iloc[:,i].value_counts()[int(ele)] )
        S = S - ( (df.iloc[:,i].value_counts()[int(ele)] ) / row)*(indi_entropy_element[int(ele)])
      else:  
        S = S - ( (df.iloc[:,i].value_counts()[ele] ) / row)*(indi_entropy_element[ele])


    elements_count = dict()
    for t in df.iloc[:,i] :
      if t not in elements_count :
        elements_count[t] = 1
      else:
        elements_count[t] =  elements_count[t] +1
    
    gain =0
    for k, p in elements_count.items():
      gain  += gain_ratio(p,len(df))
    gain = S/gain

    # print(df.columns[i])
    if(gain > val):
          node =df.columns[i]
          val = gain
          node_child = indi_entropy_element


          

  print(". "*move,node)
  for i in node_child:
    if(node_child[i] == 0.0):
      print(". "*move  , i ,"    ", "\033[1m" +ref[df[df[node] == i ].index.values[0]]+"\033[0m")
    if(node_child[i] != 0.0):
      print(" ."*move,i)
      dr = df1.loc[df1[node] == i]
      dr = dr.drop(node,axis = 1)
      dr = dr.reset_index()
      dr = dr.drop('index',axis =1)
      Input_data(dr,move+4)


In [19]:
move = 2
Input_data(mydata,move)

. .  safety
 . . high
. . . . . .  buying
. . . . . .  vhigh      unacc
 . . . . . . med
. . . . . . . . . .  maint
. . . . . . . . . .  vhigh      unacc
 . . . . . . . . . . med
. . . . . . . . . . . . . .  lug boot
. . . . . . . . . . . . . .  small      unacc
 . . . . . . . . . . . . . . med
. . . . . . . . . . . . . . . . . .  persons
 . . . . . . . . . . . . . . . . . . 4
. . . . . . . . . . . . . . . . . . . . . .  doors
. . . . . . . . . . . . . . . . . . . . . .  4      vgood
. . . . . . . . . . . . . . . . . . . . . .  3      acc
. . . . . . . . . . . . . . . . . . . . . .  5more      vgood
. . . . . . . . . . . . . . . . . . . . . .  2      acc
 . . . . . . . . . . . . . . . . . . more
. . . . . . . . . . . . . . . . . . . . . .  doors
. . . . . . . . . . . . . . . . . . . . . .  4      vgood
. . . . . . . . . . . . . . . . . . . . . .  3      vgood
. . . . . . . . . . . . . . . . . . . . . .  5more      vgood
. . . . . . . . . . . . . . . . . . . . . .  2      acc
. . . . . 